In [3]:
#Stemming words, kök almak
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()
stemmer.stem('cooking')

'cook'

In [4]:
#RegexpStemmer class, istediğimiz kısıları kelimeden silebiliriz.
from nltk.stem import RegexpStemmer
stemmer=RegexpStemmer('ing')
stemmer.stem('cooking')

'cook'

In [7]:
#Lemmatizing, WordNet ile kelimenin ağaçtaki yerine bakarak köküne ulaşabiliriz.
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
lemmatizer.lemmatize('cooking', pos='v')

'cook'

In [10]:
#yukarıdakinin aynısı farklı bir yöntemle. Farklarını gör.
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()
print(stemmer.stem('believes'))
print(lemmatizer.lemmatize('believes'))

believ
belief


In [12]:
#yukarıdakine göre stemming ve lemmatizationu birleştirelim.
print(stemmer.stem('buses'))
print(lemmatizer.lemmatize('buses'))

buse
bus


In [17]:
#regex ile kelime dönüşüm patternleri yapıyoruz, kısa yazılmış kelimeleri uzatıyoruz.
import re
replacement_patterns=[
    (r'won\'t','will not'),
    (r'can\'t','can not')
    #daha da yazılır 35. sayfada var devamı.
]

class RegexpReplacer(object):
    def __init__(self,patterns=replacement_patterns):
        self.patterns=[(re.compile(regex),rep1) for (regex,rep1) in patterns]
    
    def replace(self,text):
        s=text
        for(pattern,rep1) in self.patterns:
            s=re.sub(pattern,rep1,s)
        return s

In [18]:
#from replacers import RegexpReplacer, burada işe yaramıyor, compilerde yarar.
replacer=RegexpReplacer()
replacer.replace("can't is a constraction")

'can not is a constraction'

In [19]:
#tokenizer ile birleştirelim
from nltk.tokenize import word_tokenize
#from replacers import RegexpReplacer
replacer=RegexpReplacer()
print(word_tokenize("can't is a contraction"))#birleşmemiş hali nasıl çalışıyor?
print(word_tokenize(replacer.replace("can't is a contraction")))

['ca', "n't", 'is', 'a', 'contraction']
['can', 'not', 'is', 'a', 'contraction']


In [22]:
#Removing repeating characters için sınıf oluşturup kullanalım.
import re

class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp=re.compile(r'(\w*)(\w)\2(\w*)')
        self.rep1=r'\1\2\3'
    def replace(self,word):
        rep1_word=self.repeat_regexp.sub(self.rep1,word)
        if rep1_word!=word:
            return self.replace(rep1_word)
        else:
            return rep1_word

In [23]:
#from replacers import RepeatReplacer
replacer=RepeatReplacer()
print(replacer.replace('looooooove'))
print(replacer.replace('gooosee'))

love
gose


In [24]:
#yukarıdakinde gose hatalı bir sonuç. Daha doğru bir şey için bulunan sonuç var olan bir kelimemidir diye de
#kontrol ettirmek gerekir. Burada da wordnet kullanıyoruz.
import re
from nltk.corpus import wordnet

class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp=re.compile(r'(\w*)(\w)\2(\w*)')
        self.rep1=r'\1\2\3'
    def replace(self,word):
        if wordnet.synsets(word):
            return word
        rep1_word=self.repeat_regexp.sub(self.rep1,word)
        if rep1_word!=word:
            return self.replace(rep1_word)
        else:
            return rep1_word

In [25]:
#from replacers import RepeatReplacer buradaki sonuçlar doğru olacaklar çünkü kontrol ediliyorlar.
replacer=RepeatReplacer()
print(replacer.replace('looooooove'))
print(replacer.replace('gooosee'))

love
goose


In [ ]:
#sayfa 39 Spelling Correction With Enchant windowsta olmadığı için yapılamadı.

In [29]:
#Replacig synonyms
class WordReplacer(object):
    def __init__(self,word_map):
        self.word_map=word_map
        
    def replace(self,word):
        return self.word_map.get(word,word)
    
#from replacers import WordReplacer
replacer=WordReplacer({'bday':'birthday'})#daha da eklenebilir
print(replacer.replace('bday'))
print(replacer.replace('happy'))

birthday
happy


In [30]:
#CSV ile synonym replacement
import csv

class CsvWordReplacer(WordReplacer):
    def __init__(self,fname):
        word_map={}
        for line in csv.reader(open(fname)):
            word,syn=line
            word_map[word]=syn
        super(CsvWordReplacer,self).__init__(word_map)
        
#from replacers import CsvWordReplacer
replacer=CsvWordReplacer('synonyms.csv')
replacer.replace('bday')
#dosyayı bulamıyor.

FileNotFoundError: [Errno 2] No such file or directory: 'synonyms.csv'

In [33]:
import yaml

class YamlWordReplacer(WordReplacer):
    def __init__(self,fname):
        word_map=yaml.load(open(fname))
        super(YamlWordReplacer,self).__init__(word_map)
        
#from replacers import Yam1WordReplacer
replacer=YamlWordReplacer('synonyms.yam1')
replacer.replace('bday')

FileNotFoundError: [Errno 2] No such file or directory: 'synonyms.yam1'

In [39]:
#negatifleri pozitif olan zıtlarıyla değiştir.
from nltk.corpus import wordnet

class AntonymReplacer(object):
    def replace(self,word,pos=None):
        antonyms=set()
        for syn in wordnet.synsets(word,pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())
        if len(antonyms)==1:
            return antonyms.pop()
        else:
            return None
    
    def replace_negations(self,sent):
        i,l=0, len(sent)
        words=[]
        while i<l:
            word=sent[i]
            if word=='not' and i+1<l:
                ant=self.replace(sent[i+1])
                if ant:
                    words.append(ant)
                    i+=2
                    continue
            words.append(word)
            i+=1
        return words
    
#from replacers import AntonymReplacer
replacer=AntonymReplacer()
print(replacer.replace('good'))
print(replacer.replace('uglify'))
sent=["let's",'not','uglify','our','code']
print(replacer.replace_negations(sent))

None
beautify
["let's", 'beautify', 'our', 'code']
